In [ ]:
import csv
import numpy as np
import utils_deck_generation as idg
import healpy_pointings as hpoint
import netcdf_read_write as nrw
import utils_intensity_map as uim
from nn_plots import figure_location
from scipy.stats import qmc
import os

In [ ]:
imap_nside = 256
imap_npix = imap_nside**2 * 12

# Number of samples, size of NN training set
num_examples = 2
test_size = int(num_examples / 10)
if test_size == 0:
    test_size = 1
random_seed = 12345
run_shuffle = False

num_sim_params = 0
# pointings
surface_cover_radians = np.radians(45.0)
num_sim_params += 2
# defocus
defocus_range = 20.0 # mm)
num_sim_params += 1
#power
min_power = 0.5 # fraction of full power
num_sim_params += 1

run_sims = True
run_compression = True
run_clean = True

run_type = "nif" #"test" #"nif"
num_quads = 48
num_cones = 8

run_dir = "Data"

In [ ]:
filename_trainingdata = run_dir + "/" + "training_data_and_labels.nc"
LMAX = 30
num_coeff = int(((LMAX + 2) * (LMAX + 1))/2.0)
# Assume symmetry
num_output = int(num_cones/2) * num_sim_params
print(num_coeff*2, num_examples, num_output)

In [ ]:
rng = np.random.default_rng(random_seed)
sampler = qmc.LatinHypercube(d=num_output, seed=rng)
sample = sampler.random(n=num_examples)
Y_train = sample.T
print(np.shape(Y_train))

## Import NIF: Beam, Quad and Cone data

In [ ]:
run_data = dict()

run_data['nbeams'] = 192
run_data['target_radius'] = 1100.0
run_data['facility'] = "NIF"
run_data['num_quads'] = num_quads
run_data['default_power'] = 0.25 #TW per beam

In [ ]:
filename = "NIF_UpperBeams.txt"
j = -1

f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j==-1:
        key = row
        for i in range(len(row)):
            run_data[row[i]] = [None] * int(run_data['nbeams'])
    else:
        for i in range(len(row)):
            if i < 2:
                run_data[key[i]][j] = row[i]
            elif i < 5:
                run_data[key[i]][j] = float(row[i])
            else:
                run_data[key[i]][j] = int(row[i])
    j=j+1
f.close()
filename = "NIF_LowerBeams.txt"
f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j>96:
        for i in range(len(row)):
            if i < 2:
                run_data[key[i]][j-1] = row[i]
            elif i < 5:
                run_data[key[i]][j-1] = float(row[i])
            else:
                run_data[key[i]][j-1] = int(row[i])
    j=j+1
f.close()

run_data["Theta"] = np.radians(run_data["Theta"])
run_data["Phi"] = np.radians(run_data["Phi"])

In [ ]:
run_data['quad_from_each_cone'] = ('Q15T', 'Q13T', 'Q14T', 'Q11T', 'Q15B', 'Q16B', 'Q14B', 'Q13B')

# Generate and Save Training Data

In [ ]:
coord_o = np.zeros(3)
coord_o[2] = run_data['target_radius']
run_data['pointings'] = np.zeros((run_data['nbeams'], 3))
run_data["Port_centre_theta"] = np.zeros(run_data['nbeams'])
run_data["Port_centre_phi"] = np.zeros(run_data['nbeams'])
run_data["defocus"] = np.zeros(run_data['nbeams'])
run_data["p0"] = np.zeros(run_data['nbeams'])
run_data["fuse"] = [False] * run_data["nbeams"]
run_data['beams_per_cone'] = [0] * num_cones

sim_params = np.zeros((num_output*2, num_examples))
theta_pointings = np.zeros((run_data["nbeams"], num_examples))
phi_pointings = np.zeros((run_data["nbeams"], num_examples))

X_train = np.zeros((num_coeff * 2, num_examples))
avg_powers = np.zeros(num_examples)

for iex in range(num_examples):
    ex_params =  Y_train[:,iex]
    for icone in range(num_cones):
        il = (icone*4) % num_output
        iu = ((icone+1)*4-1) % num_output + 1
        cone_params = ex_params[il:iu]

        x = cone_params[0] * 2.0 - 1.0
        y = cone_params[1] * 2.0 - 1.0
        r, offset_phi = hpoint.square2disk(x, y)
        if icone > 3:
            offset_phi = (offset_phi + np.pi) % (2.0 * np.pi)
        offset_theta = r * surface_cover_radians

        cone_defocus = cone_params[2] * defocus_range # convert to mm
        cone_power = cone_params[3] * (1.0 - min_power) + min_power

        quad_name = run_data['quad_from_each_cone'][icone]
        quad_start_ind = run_data["Quad"].index(quad_name)
        quad_slice = slice(quad_start_ind, quad_start_ind+4)

        cone_name = run_data['Cone'][quad_slice]
        cone_name = cone_name[0]
        
        sim_params[icone*4:(icone+1)*4,iex] = (offset_theta, offset_phi, cone_defocus, cone_power)

        # divide by 2 so only uppers or lowers
        beams_in_cone_count = int(run_data["Cone"].count(cone_name)/2)
        ind = run_data["Quad"].index(quad_name)
        # 4 beams per quad so set skip = 4
        cone_slice = slice(ind,ind+beams_in_cone_count,4)
        quad_list_in_cone = run_data["Quad"][cone_slice]
        run_data['beams_per_cone'][icone] = beams_in_cone_count

        for quad_name in quad_list_in_cone:
            ind = run_data["Quad"].index(quad_name)
            quad_slice = slice(ind,ind+4)
            beam_names = run_data['Beam'][quad_slice]

            run_data["Port_centre_theta"][quad_slice] = np.mean(run_data["Theta"][quad_slice])
            run_data["Port_centre_phi"][quad_slice] = np.mean(run_data["Phi"][quad_slice])
            port_theta = run_data["Port_centre_theta"][ind]
            port_phi = run_data["Port_centre_phi"][ind]
            
            rotation_matrix = np.matmul(np.matmul(hpoint.rot_mat(port_phi, "z"), hpoint.rot_mat(port_theta, "y")),
                              np.matmul(hpoint.rot_mat(offset_phi, "z"), hpoint.rot_mat(offset_theta, "y")))

            coord_n = np.matmul(rotation_matrix, coord_o)
            
            theta_pointings[quad_slice,iex] = np.arccos(coord_n[2] / run_data['target_radius'])
            phi_pointings[quad_slice,iex] = np.arctan2(coord_n[1], coord_n[0])

            run_data['pointings'][ind:ind+4,:] = np.array(coord_n)
            run_data["defocus"][quad_slice] = cone_defocus
            run_data["p0"][quad_slice] = run_data['default_power'] * cone_power

    run_location = run_dir + "/" + "run_" + str(iex)
    if run_sims:
        idg.generate_input_deck(run_data, run_location)
        idg.generate_input_pointing_and_pulses(run_data, run_location, run_type)

        b='Running ifriit in: ' + run_location + "  "
        print("\r", b, end="")
        !cd $run_location;chmod +x main;mpirun -np 1 ./main > a.out

    if run_compression:
        intensity_map = nrw.read_intensity(run_location, run_type, run_data['Beam'], imap_nside)
        X_train1, avg_power1 = uim.create_xtrain(intensity_map, LMAX)

        X_train[:,iex] = X_train1
        avg_powers[iex] = avg_power1

    if run_clean:
        os.remove(run_location + '/main')
        for beam in run_data['Beam']:
            os.remove(run_location + '/p_in_z1z2_beam_NIF-' + beam + '.nc')

if run_compression:
    nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

## Diagnostic

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp

In [ ]:
iex = 1

run_location = run_dir + "/" + "run_" + str(iex)
theta_slice = slice(0,29,4)
phi_slice   = slice(1,30,4)
power_slice = slice(3,32,4)

cone_theta_offset = sim_params[theta_slice,iex]
cone_phi_offset = sim_params[phi_slice,iex]
cone_powers = sim_params[power_slice,iex]

beams_prev = 0
beams_tot = 0
total_power = 0
for icone in range(num_cones):
    beams_per_cone = run_data['beams_per_cone'][icone]
    beams_tot += beams_per_cone
    total_power += cone_powers[icone] * beams_per_cone
    beams_prev += beams_per_cone
mean_power_fraction = total_power / run_data['nbeams']

intensity_map = nrw.read_intensity(run_location, run_type, run_data['Beam'], imap_nside)
intensity_map_rms_spatial = uim.readout_intensity(run_data, intensity_map, mean_power_fraction)

hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = run_data["Port_centre_theta"]
port_phi = run_data["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(theta_pointings[:,iex], phi_pointings[:,iex])

## Incident fraction, RMS and modes

In [ ]:
import utils_intensity_map as uim

In [ ]:
hp.mollview(intensity_map_rms_spatial, unit="Deviation from mean (%)",flip="geo")
plt.savefig(figure_location+'/deviation_from_mean_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
LMAX = 30
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes LLE")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/unweighted_modes.png", dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/weighted_modes.png", dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview(intensity_map_normalized, unit="Deviation from Mean", flip="geo")

X_train = uim.imap2xtrain(intensity_map_normalized, LMAX, avg_power)
intensity_map_normalized2 = uim.xtrain2imap(X_train, LMAX, imap_nside, avg_power)
hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

# Create data structure for data set and label

## Load Data Set

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import healpy as hp

In [ ]:
training_data = Dataset(filename_trainingdata)
X_all = training_data.variables["X_train"][:]
Y_all = training_data.variables["Y_train"][:]
avg_powers_all = training_data.variables["avg_powers"][:]
training_data.close()

print(np.shape(X_all), np.shape(Y_all))

## Shuffle and Seperate Data

In [ ]:
if run_shuffle:
    index_shuf = list(range(num_examples))
    rng = np.random.default_rng(random_seed)
    rng.shuffle(index_shuf)
    index_shuf = np.array(index_shuf)
    X_train = []
    Y_train = []
    train_avg_powers = []
    X_test = []
    Y_test = []
    test_avg_powers = []
    j=0
    for i in index_shuf:
        if (j < test_size):
            X_test.append(X_all[:,i])
            Y_test.append(Y_all[:,i])
            test_avg_powers.append(avg_powers_all[i])
        else:
            X_train.append(X_all[:,i])
            Y_train.append(Y_all[:,i])
            train_avg_powers.append(avg_powers_all[i])
        j = j + 1
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
else:
    X_test = X_all.T[:test_size,:]
    Y_test = Y_all.T[:test_size,:]
    test_avg_powers = avg_powers_all[:test_size]

    X_train = X_all.T[test_size:,:]
    Y_train = Y_all.T[test_size:,:]
    train_avg_powers = avg_powers_all[test_size:]

print(np.shape(X_train), np.shape(Y_train), np.shape(X_test), np.shape(Y_test))

## Check Shuffle Maintained labelling

In [ ]:
if run_shuffle:
    ind = 0
    ind_first = np.where(index_shuf == ind)
    ind_first = np.squeeze(ind_first[0])

    print(Y_all[:,ind])
    intensity_map_normalized2 = uim.xtrain2imap(X_all[:,ind], LMAX, imap_nside, avg_powers_all[ind])
    hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

    if ind_first >= test_size:
        i = ind_first-test_size
        print(Y_train[i,:])
        cdata = X_train[i,:]
        apwrs = train_avg_powers[i]
    else:
        i = ind_first
        print(Y_test[i,:])
        cdata = X_test[i,:]
        apwrs = test_avg_powers[i]
    intensity_map_normalized2 = uim.xtrain2imap(cdata, LMAX, imap_nside, apwrs)
    hp.mollview(intensity_map_normalized2, unit="Deviation from Mean",flip="geo")

## Normalize

In [ ]:
mu = np.mean(X_train)
sigma = np.std(X_train)

X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

print(np.std(X_train), np.std(X_test))

# Neural Network

In [ ]:
import tf_neural_network as tfnn
import matplotlib.pyplot as plt
import nn_plots as nnp

In [ ]:
filename_nn_weights = "neural_network_weights"
input_size = num_coeff * 2
output_size = num_output
lr = 0.001
num_epochs = 100
costs = []
train_acc = []
test_acc = []
epochs = []
hidden_units1 = 25
hidden_units2 = 20

In [ ]:
parameters, costs, train_acc, test_acc, epochs = tfnn.model_wrapper(X_train, Y_train, X_test, Y_test, learning_rate = lr, num_epochs = num_epochs, hidden_units1=hidden_units1, hidden_units2=hidden_units2)
nrw.save_nn_weights(parameters, filename_nn_weights)

In [ ]:
nnp.plotting(epochs, costs, train_acc, test_acc, lr)

## Open NN Weights

In [ ]:
parameters = nrw.read_nn_weights(filename_nn_weights)

## Restart Training

In [ ]:
start_epoch = num_epochs
num_epochs = 1000

parameters, costs2, train_acc2, test_acc2, epochs2 = tfnn.model_wrapper(X_train, Y_train, X_test, Y_test, learning_rate = lr, num_epochs = num_epochs, start_epoch = start_epoch, nn_weights = parameters)
nrw.save_nn_weights(parameters, filename_nn_weights)

costs = np.append(costs, costs2)
train_acc = np.append(train_acc, train_acc2)
test_acc = np.append(test_acc, test_acc2)
epochs = np.append(epochs, epochs2)

In [ ]:
nnp.plotting(epochs, costs, train_acc, test_acc, lr)

## Apply NN

In [ ]:
ie = 0

x_test = np.reshape(X_test[ie,:], (-1,input_size))
y_true = np.squeeze(Y_test[ie,:])

y_pred = tfnn.apply_network(x_test, parameters)
y_pred = np.squeeze(y_pred)
print(y_pred)
print(y_true)
print((np.abs(y_true - y_pred)) / y_true * 100)
print(np.mean((np.abs(y_true - y_pred)) / y_true * 100))